Basic Classification on Fashion MNIST dataset

In [1]:
!pip install -U tensorflow_datasets

     |████████████████████████████████| 4.2 MB 12.0 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [ ]:
class_names = metadata.features['label'].names
class_names

In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_train_examples

In [ ]:
num_test_examples = metadata.splits['test'].num_examples
num_test_examples

**Preprocess**

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

**Explore**

In [ ]:
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape(28,28)

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(train_dataset.take(20)):
  image = image.numpy().reshape(28,28)
  plt.subplot(5,4,i+1)
  plt.xlabel(class_names[label])
  plt.imshow(image)
plt.show()

**Build the model**

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(128, activation=tf.nn.relu),
                             tf.keras.layers.Dense(10, activation=tf.nn.softmax)                        
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [14]:
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [15]:
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/5
1875/1875 [==============================] - 14s 3ms/step - loss: 0.4927 - accuracy: 0.8288
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3742 - accuracy: 0.8655
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3416 - accuracy: 0.8760
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3121 - accuracy: 0.8848
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3009 - accuracy: 0.8916


**Evaluation**

In [16]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

313/313 [==============================] - 4s 11ms/step - loss: 0.3468 - accuracy: 0.8744
Accuracy on test dataset: 0.8744000196456909
